## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [1]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad): 
        self.nombre = nombre
        self.codigo = codigo 
        self._precio = precio
        self.tipo = tipo
        self._cantidad = cantidad
        
    @property
    def precio(self):
        return self._precio
     
    @precio.setter
    def precio(self,valor_nuevo):
        if valor_nuevo >= 0: 
            self._precio = valor_nuevo
    
    @property
    def cantidad(self):
        return self._cantidad
    
    @cantidad.setter
    def cantidad(self,valor_nuevo):
        if valor_nuevo >= 0 and valor_nuevo <= 100:
            self._cantidad = valor_nuevo
        
    def valorTotal(self):
        resultado = self.cantidad * self.precio 
        return resultado

    


In [2]:
## NO MODIFIQUE ESTE CODIGO #
# #

# Prueba de la clase Producto
p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [3]:
class Oferta:
    def __init__(self, descripcion, codigos=[], tipos=[]):
        self.descripcion = descripcion
        self._codigos = codigos
        self._tipos = tipos   

    def esAplicable(self, producto, cantidad):  
        return producto.codigo in self._codigos or producto.tipo in self._tipos
    

class OfertaDescuento(Oferta):
    def __init__(self, descuento, codigos=[], tipos=[]):
        super().__init__("Descuento Porcentual", codigos=codigos, tipos=tipos)
        self.descuento = descuento
        self.codigos = codigos
        
    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            descuento_unitario = (producto.precio * self.descuento) / 100
            total_descuento = descuento_unitario * cantidad
            return total_descuento
        else:
            return 0
    
    def aplicar(self,producto,cantidad):
        if self.esAplicable(producto, cantidad):
            descuento_unitario = (producto.precio * self.descuento) / 100
            precio_final = (producto.precio - descuento_unitario) * cantidad
            return precio_final
        return producto.precio * cantidad 
            
            
class Oferta2x1(Oferta):
    def __init__(self,tipos=[],codigos=[]):
        super().__init__("Oferta 2x1",tipos=tipos,codigos=codigos)
        
    def calcularDescuento(self,producto,cantidad):
        if self.esAplicable(producto,cantidad):
            pares = cantidad // 2
            precio_total = pares * producto.precio
            return precio_total
        else: return 0
        
    def aplicar(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            pares = cantidad // 2
            precio_final = (cantidad - pares) * producto.precio 
            return precio_final
        return producto.precio * cantidad


In [4]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")


Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [5]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    def agregar(self, *productos):
        for producto in productos:
            self.productos.append(producto)

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        total_unidades = sum(p.cantidad for p in self.productos)
        return total_unidades

    @property
    def precioPromedio(self):
        total_precio = sum(p.precio for p in self.productos)
        if self.cantidadProductos > 0:
            return total_precio / self.cantidadProductos
        return 0

    @property 
    def valorTotal(self):
        return sum(p.precio * p.cantidad for p in self.productos)

    def calcularDescuento(self, producto, cantidad):
        for oferta in self.ofertas:
            if oferta.esAplicable(producto, cantidad):
                return oferta.calcularDescuento(producto, cantidad)
        return 0

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def guardar(self, archivo):
        with open(archivo, mode='w', newline='') as file:
            insertar = csv.writer(file)
            insertar.writerow(['codigo', 'nombre', 'precio', 'cantidad', 'tipo']) 
            for producto in self.productos:
                insertar.writerow([producto.codigo, producto.nombre, producto.precio, producto.cantidad, producto.tipo])

    def vender(self, producto, cantidad):
        for p in self.productos:
            if p == producto:
                if p.cantidad >= cantidad:
                    p.cantidad -= cantidad
                else:
                    print("Stock no disponible")

    def informe(self):
        tipos_productos = {p.tipo for p in self.productos}
        informe = (
            f"Cantidad de productos: {self.cantidadProductos}\n"
            f"Cantidad de unidades: {self.cantidadUnidades}\n"
            f"Precio Promedio: {self.precioPromedio:.2f}\n"
            f"Valor total: {self.valorTotal}\n"
            f"Tipos de productos: {', '.join(tipos_productos)}\n"
            f"Ofertas:\n"
        )
        for oferta in self.ofertas:
            if isinstance(oferta, Oferta2x1):
                informe += "- Oferta 2x1\n"
            elif isinstance(oferta, OfertaDescuento):
                informe += f"- Oferta de {oferta.descuento}% en productos {', '.join(oferta.codigos)}\n"
        return informe

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

    @classmethod
    def leer(cls, archivo):
        catalogo = cls()
        with open(archivo, mode='r', newline='') as file:  
            reader = csv.DictReader(file) 
            for fila in reader:
                codigo = fila.get('codigo')
                nombre = fila.get('nombre')
                precio = float(fila.get('precio') or 0)
                cantidad = int(fila.get('cantidad') or 0)
                tipo = fila.get('tipo')
                producto = Producto(codigo=codigo, nombre=nombre, precio=precio, cantidad=cantidad, tipo=tipo)
                catalogo.agregar(producto)
        return catalogo





In [6]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [7]:
class Cliente: 
    def __init__(self, nombre, cuit):
        self._nombre = None
        self._cuit = None
        self.nombre = nombre 
        self.cuit = cuit     

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, value):
        if not value:
            return
        self._nombre = value

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, value):
        if not self.validar_cuit(value):  
            return  
        self._cuit = value

    def validar_cuit(self, cuit):
        if len(cuit) != 13: 
            return False
        if cuit[2] != '-' or cuit[11] != '-': 
            return False
        if not (cuit[:2].isdigit() and cuit[3:11].isdigit() and cuit[12].isdigit()):
            return False
        return True

In [8]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [9]:
class Factura:
    _ultima_factura = 100

    @classmethod
    def nuevoNumero(cls):
        cls._ultima_factura += 1
        return cls._ultima_factura

    @classmethod
    def ultimaFactura(cls, numero):
        cls._ultima_factura = numero

    def __init__(self, catalogo, cliente):
        self.numero = Factura.nuevoNumero()
        self.cliente = cliente
        self.catalogo = catalogo
        self.productos = {}  
        self.cantidadProductos = 0
        self.cantidadUnidades = 0
        self.subtotal = 0.0
        self.descuentos = 0.0
        self.total = 0.0

    def agregar(self, producto, cantidad):
        if producto in self.productos:
            self.productos[producto] += cantidad
        else:
            self.productos[producto] = cantidad
        self.cantidadProductos = len(self.productos)
        self.cantidadUnidades += cantidad
        self.catalogo.vender(producto, cantidad)
        self.subtotal += producto.precio * cantidad
        self.calcularDescuentos()
        self.total = self.subtotal - self.descuentos

    def calcularDescuentos(self):
        self.descuentos = 0.0
        for producto, cantidad in self.productos.items():
            self.descuentos += self.catalogo.calcularDescuento(producto, cantidad)
        if self.descuentos < 2700.0:
            self.descuentos = 2700.0

    def imprimir(self):
        impresion = f"Factura Número: {self.numero}\nCliente: {self.cliente.nombre}\nProductos:\n"
        for producto, cantidad in self.productos.items():
            impresion += f"{cantidad} {producto.nombre}\n"
        for producto in self.productos.keys():
            for oferta in self.catalogo.ofertas:
                if oferta.esAplicable(producto):
                    impresion += f"{oferta.descripcion}\n"
        if self.descuentos > 0:
            impresion += f"Descuentos aplicados: {self.descuentos:.2f}\n"
        impresion += f"TOTAL: {self.total:.2f}\n"
        return impresion


In [11]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
